In [1]:
import numpy as np
# s = open("19.txt").read().splitlines()
# s.append("")

# lines = []

# acc = []

# for l in s:

#     if l=="":
#         acc.append(np.array([[int(i) for i in b.split(",")] for b in lines[1:]]))
#         lines = []
#     else:
#         lines.append(l)


In [2]:
def rotate_x_y(x):

    x = x[:,[1,0,2]]

    x[:,0]*=-1

    return x

def rotate_x_z(x):

    x = x[:,[2,1,0]]

    x[:,0]*=-1

    return x

def rotate_y_z_1(x):

    x = x[:,[0,2,1]]

    x[:,2]*=-1

    return x

def rotate_y_z_2(x):

    x = x[:,[0,2,1]]

    x[:,1]*=-1

    return x


def rotate_all(x):


    # first y_z

    for func in [rotate_y_z_1,rotate_y_z_2]:
        
        y = func(x)

        yield y

        for _ in range(3):

            y = rotate_x_y(y) 
            yield y

    for i in range(4):

        yield (x)

        z = x

        for _ in range(3):

            z = rotate_x_y(z)
            yield z
        
        x = rotate_x_z(x)

In [3]:
# res = list(rotate_all(acc[0]))
# d = {i:0 for i in range(24)}
# for i,a in enumerate(res):
#     for b in res:
#         if np.allclose(a,b):
#             d[i]+=1

# assert all([i==1 for i in d.values()])

In [4]:
def rel_dis_fast(x):

    m = (np.expand_dims(x,0)- np.expand_dims(x,1))

    mask =(np.abs(m)<= 3000).all(-1)
    mask[np.arange(len(m)),np.arange(len(m))] = False
    return m[mask]
    

In [5]:
def match(a,rels):

    for k in rotate_all(a):
        
        rel_k = rel_dis_fast(k)

        n_same = (np.expand_dims(rel_k ,0) == np.expand_dims(rels,1)).all(axis=-1).sum()

        if n_same >= 66:
            return k

    

In [6]:
# rels = rel_dis_fast(acc[0])
# match(acc[1],rels)

In [7]:
import numpy as np
import aocd
import sys
import time
start = time.time()

def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush() 

s = aocd.get_data(day=19,year=2021)
s=[c.splitlines()[1:] for c in s.split("\n\n")]
s = [[eval(b) for b in d ] for d in s]
acc = [np.array(i) for i in s]
acc[0].shape


(26, 3)

In [8]:
def dirty_l2(x):

    x = ((np.expand_dims(x,0)- np.expand_dims(x,1))**2).sum(-1).reshape(-1) 
    return np.unique(x)

l2s = [dirty_l2(x) for x in acc]

matches = {i:set() for i in range(len(l2s))}
for i in range(len(l2s)):
    for b in range(i+1,len(l2s)):
        
        n_same = len(set(l2s[i]).intersection(set(l2s[b])))
        if n_same >=32:
            
            matches[i].add(b)
            matches[b].add(i)

matches


{0: {13},
 1: {16, 18, 22, 24, 38},
 2: {11, 28},
 3: {30},
 4: {10, 17, 23},
 5: {7, 34},
 6: {14, 19, 29},
 7: {5, 20, 25},
 8: {27},
 9: {21},
 10: {4, 24},
 11: {2, 13, 15, 20, 25},
 12: {17, 21},
 13: {0, 11, 26},
 14: {6, 37},
 15: {11, 21, 26},
 16: {1, 37},
 17: {4, 12, 19, 22, 24, 35},
 18: {1, 27, 31},
 19: {6, 17, 23, 37},
 20: {7, 11, 21},
 21: {9, 12, 15, 20, 32, 35},
 22: {1, 17},
 23: {4, 19, 33},
 24: {1, 10, 17, 37},
 25: {7, 11, 28},
 26: {13, 15},
 27: {8, 18, 38},
 28: {2, 25},
 29: {6, 30},
 30: {3, 29},
 31: {18, 36},
 32: {21},
 33: {23},
 34: {5},
 35: {17, 21},
 36: {31},
 37: {14, 16, 19, 24},
 38: {1, 27}}

In [9]:

scanners = [np.array(b) for b in s]
scanner_pos = [(0,0,0)]



def get_scanner_pos(c,beacons):
    for b in beacons:
        r = [np.add(b,-1*np.array(c)).tolist() for c in beacons if b!=c]
        for q in c:
            if len([True for j in c if [q[0]-j[0],q[1]-j[1],q[2]-j[2]] in r])>=11:
                return [b[0]-q[0],b[1]-q[1],b[2]-q[2]]

center = scanners[0]
done = [0]
to_do = [(0,c) for c in matches[0]]
crd = rel_dis_fast(center)
belongs_to = {0:[i for i in range(len(center))]}

for i in range(1,len(scanners)):
    belongs_to[i]=[]
while to_do:
    
    relevant, n = to_do.pop()

    beacons = center[belongs_to[relevant]]
    crd = rel_dis_fast(beacons)
    c = scanners[n]
    nc = match(c,crd)
    c_pos = get_scanner_pos(nc.tolist(),beacons.tolist())
    scanner_pos.append(c_pos)
    nc[:]+=c_pos
    for b in nc.tolist():
        STOP = False
        for pos,p in enumerate(center.tolist()):
            if b==p:
                belongs_to[n].append(pos)
                STOP = True
                break
        if STOP:
            continue
        belongs_to[n].append(len(center))
        center = np.concatenate((center,[b]))        
    done.append(n)
    for t in matches[n]:
        if t not in done and t not in [b for a,b in to_do]:
            to_do.append((n,t))
    progress(len(done),len(scanners),"Merging beacons")

print("\nPart 1", len(center))

k = 0
for n in scanner_pos:
    for j in scanner_pos:
        if n!=j:
            k = max(k,sum([abs(a-b) for a,b in zip(n,j)]))
print("Part 2",k)
print("Took ",time.time()-start)

[============================================================] 100.0% ...Merging beacons
Part 1 442
Part 2 11079
Took  4.035885810852051


In [10]:
import numpy as np
import aocd
import sys
import time
start = time.time()

def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush() 

s = aocd.get_data(day=19,year=2021)
s=[c.splitlines()[1:] for c in s.split("\n\n")]
s = [[eval(b) for b in d ] for d in s]

def hashes(scanner):
    h = []
    for i,b in enumerate(scanner):
        for y,d in enumerate(scanner):
            if i!=y:
                h.append(np.sqrt(((b-d)**2).sum()))
    return np.unique(h)

h_s = [hashes(np.array(b)) for b in s]

matches = {i:set() for i in range(len(s))}
for q,w in enumerate(h_s):
    for e,r in enumerate(h_s):
        if q!=e:
            if len(set(r).intersection(w))>=32:
                matches[q].add(e)
                matches[e].add(q)

def rotate_x_y(ar,d="xy"):
    if d=="xy":
        ar= ar[:,[1,0,2]]
        ar[:,0]*=-1    
    elif d=="xz":
        ar= ar[:,[2,1,0]]
        ar[:,0]*=-1    
    elif d[:2]=="yz":
        ar= ar[:,[0,2,1]]
        ar[:,2]*=-1 * (1-2*(d[2]=="1"))
        ar[:,1]*= (1-2*(d[2]=="1"))
    return ar

scanners = [np.array(b) for b in s]
scanner_pos = [(0,0,0)]


def rel_dis(scanner):
    h = []
    for i,b in enumerate(scanner):
        for o,d in enumerate(scanner):
            if i!=o:
                iii = np.add(d,b*-1)
                if np.abs(iii).max()<=3000:
                    h.append(iii.tolist())
    return h

def match(c):
    global crd
    ll = 0
    while ll<6:
        if ll<2:
            q = rotate_x_y(c,f"yz{ll+1}")
            for i in range(4):
                if len([True for b in rel_dis(q) if b in crd])>=66:
                    return q
                q = rotate_x_y(q)
        else:
            q = c
            for i in range(4):
                if len([True for b in rel_dis(q) if b in crd])>=66:
                    return q
                q = rotate_x_y(q)
            c = rotate_x_y(c,"xz")
        ll+=1

def get_scanner_pos(c,beacons):
    for b in beacons:
        r = [np.add(b,-1*np.array(c)).tolist() for c in beacons if b!=c]
        for q in c:
            if len([True for j in c if [q[0]-j[0],q[1]-j[1],q[2]-j[2]] in r])>=11:
                return [b[0]-q[0],b[1]-q[1],b[2]-q[2]]

center = scanners[0]
done = [0]
to_do = [(0,c) for c in matches[0]]
crd = rel_dis(center)
belongs_to = {0:[i for i in range(len(center))]}

for i in range(1,len(scanners)):
    belongs_to[i]=[]
while to_do:
    
    relevant, n = to_do.pop()

    beacons = center[belongs_to[relevant]]
    crd = rel_dis(beacons)
    c = scanners[n]
    nc = match(c)
    c_pos = get_scanner_pos(nc.tolist(),beacons.tolist())
    scanner_pos.append(c_pos)
    nc[:]+=c_pos
    for b in nc.tolist():
        STOP = False
        for pos,p in enumerate(center.tolist()):
            if b==p:
                belongs_to[n].append(pos)
                STOP = True
                break
        if STOP:
            continue
        belongs_to[n].append(len(center))
        center = np.concatenate((center,[b]))        
    done.append(n)
    for t in matches[n]:
        if t not in done and t not in [b for a,b in to_do]:
            to_do.append((n,t))
    progress(len(done),len(scanners),"Merging beacons")

print("\nPart 1", len(center))

k = 0
for n in scanner_pos:
    for j in scanner_pos:
        if n!=j:
            k = max(k,sum([abs(a-b) for a,b in zip(n,j)]))
print("Part 2",k)
print("Took ",time.time()-start)

[============================================================] 100.0% ...Merging beacons
Part 1 442
Part 2 11079
Took  4.103910446166992
